In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm

In [2]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def CSHam_ex(N, B, Ak, GS, Filling):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # now we add the extra gap filler
    #GS_s = scipy.sparse.csr_matrix(GS)
    #fill_Op = Filling * scipy.sparse.kron(GS_s, scipy.sparse.spmatrix.getH(GS_s))
    fill_Op = Filling * np.outer(GS, np.conj(GS))
    operators.append(fill_Op)
    sites.append(np.arange(0,N).tolist())
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr

In [11]:
# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.05)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.01) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.variational.MCState(self.sampler, self.machine, n_samples=1000, n_discard=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output):
        self.vs.init_parameters(nk.nn.initializers.normal(stddev=0.25))
        gs = nk.VMC(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, preconditioner=self.sr)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=1000, show_progress = False)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, runTime
    
# Combines all steps into a function to run on the cluster
def runDescentCS_ex(N,B,Ak,alpha,GS,Filling, i):
    np.random.seed()
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam_ex(N,B,Ak,GS,Filling)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/excited/ex_Logs"+str(N)+"run"+str(i)+"filling"+str(round(Filling,2))) #where _call_ will be invoked
    return eng, state, runTime

# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha):
    np.random.seed()
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, runTime = rbm("2021_summer_data/excited/Logs"+str(N)) #where _call_ will be invoked
    return eng, state, runTime

In [5]:
# Here we find the different values of gap filling for testing
Ak = []
N = 5
B=0.95
# Variable A
#A = N/2
#N0 = N/2
for i in range(4):
        # Constant A
    Ak_i = 1
        # Variable A
        #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)

ha, hi = CSHam(N, B, Ak)

e,v  = exactDiagonalization_full(ha)
print('exact ground state is',e)
fill_1 = np.abs(e[1]-e[0])
fill_2 = fill_1 + 0.5 * np.abs(e[2]-e[1])
fill_3 = fill_1 + np.abs(e[2]-e[1])
fill_4 = np.abs(e[0]) + np.abs(e[-1]) + 1

filling = [fill_1, fill_2, fill_3, fill_4]
print(filling)

exact ground state is [-1.83133646 -1.67324453 -1.49524094 -1.28712343 -1.26273146 -1.26273146
 -1.26273146 -0.9920411  -0.9920411  -0.9920411  -0.475      -0.475
  0.025       0.025       0.025       0.475       0.475       0.4920411
  0.4920411   0.4920411   0.525       0.76273146  0.76273146  0.76273146
  0.78712343  0.975       0.975       0.975       0.99524094  1.17324453
  1.33133646  1.475     ]
[0.1580919273378183, 0.2470937234417182, 0.3360955195456181, 4.306336460086846]


In [6]:
N = 5
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
#A = N/2
#N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization(ha)
#Ground state energy
edEng = e[0]
# Ground state
edState = v[0]

#Lists for Histogram Data
numRuns = 1
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []
GS = []

# Get errors for each run in histogram
for i in range(len(hisIt)):
    engTemp, stateTemp, runTimeTemp = runDescentCS(N,B,Ak,alpha)
    #engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    GS.append(stateTemp)
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)
print(GS)


Eng error  [0.014016405201813242]
State error  [0.0694505401121156]
[DeviceArray([-2.08699870e-02+1.06246904e-02j,
             -1.08212134e-01+1.15548130e-01j,
             -1.10063062e-01+1.16086707e-01j,
             -2.67233568e-02+4.98887042e-02j,
             -1.08516927e-01+1.15863892e-01j,
             -2.32762585e-02+4.93654339e-02j,
             -2.26956544e-02+4.63740538e-02j,
             -5.66409924e-03+2.72319872e-03j,
             -1.08386131e-01+1.14837322e-01j,
             -2.90841032e-02+5.09121287e-02j,
             -2.44053970e-02+4.99453097e-02j,
             -3.98610926e-03+9.77653080e-03j,
             -2.74410634e-02+4.87698379e-02j,
             -4.74178528e-03+7.61158363e-03j,
             -7.64770429e-03+1.01463243e-03j,
             -1.48961505e-03-2.96581569e-03j,
              6.22403672e-01-6.65150681e-01j,
              5.14157489e-02-9.81160652e-02j,
              5.69540439e-02-9.81913959e-02j,
              4.86580416e-03-7.12059244e-03j,
           

In [16]:
N = 2
alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
#B=N/2
#A = N/2
#N0 = N/2
# List of Ak
Ak = []
for i in range(N - 1):
    # Constant A
    Ak_i = 1
    # Variable A
    #Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
ha, hi = CSHam(N,B,Ak)    
e,v  = exactDiagonalization_full(ha)

#Ground state energy
edEng = e[1]
# Ground state
edState = v[1]





In [17]:


Fill = filling[0]

#Lists for Histogram Data
numRuns = 5
hisIt = np.arange(numRuns)
engErr = []
stateErr = []
runTime = []

# Cluster multiproccessing
ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=10))
pool = mp.Pool(processes=ncpus)
# Run Descent
results = []
for j in hisIt:
    result = pool.apply_async(runDescentCS_ex, args=(N,B,Ak,alpha, GS, Fill, j)) 
    results.append(result)

pool.close()
pool.join()
resultsSR = [r.get() for r in results]





# Get errors for each run in histogram
for i in range(len(hisIt)):
    #engTemp, stateTemp, runTimeTemp = runDescentCS_ex(N,B,Ak,alpha, GS[0], filling[f_i], i)
    engTemp, stateTemp, runTimeTemp = resultsSR[i]
    runTime.append(runTimeTemp)
    errSR = err(np.asmatrix(stateTemp), edState, engTemp, edEng,N) #make state vector as matrix data-type
    engErr.append(errSR[0])
    stateErr.append(errSR[1])
    print('Eng error ', engErr)
    print('State error ', stateErr)

#Save data to JSON file
data = [engErr, stateErr, runTime]
fileName = "2021_summer_data/excited/con_N"+str(N)+"M" + str(M)+"fill"+str(round(Fill,2))+".json"
open(fileName, "w").close()
with open(fileName, 'a') as file:
    for item in data:
        line = json.dumps(item)
        file.write(line + '\n')
print('SAVED')


Process ForkPoolWorker-71:
Process ForkPoolWorker-75:
Process ForkPoolWorker-74:
Process ForkPoolWorker-73:
Process ForkPoolWorker-72:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.2/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.2/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.2/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.2/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.2/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/pytho